In [1]:
print('jai maa')

jai maa


In [17]:
import numpy as np
import pandas as pd
import torch
import os, cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transform
# Input data files are available in the read-only "../input/" directory

directory_path = 'D:\\python\\python_2\\data'   
# data_dir = os.listdir(directory_path)
# print(data_dir)
fake_data = os.listdir(directory_path +'/fake data')
real_data = os.listdir(directory_path +'/real data')
print(real_data)

['0000.mp4', '0001.mp4', '0002.mp4', '0003.mp4', '0004.mp4', '0005.mp4', '0006.mp4', '0007.mp4', '0008.mp4', '0009.mp4', '0010.mp4', '0011.mp4', '0012.mp4', '0013.mp4', '0014.mp4', '0015.mp4', '0016.mp4', '0017.mp4', '0018.mp4', '0019.mp4', '0020.mp4', '0021.mp4', '0022.mp4', '0023.mp4', '0024.mp4', '0025.mp4', '0026.mp4', '0027.mp4', '0028.mp4', '0029.mp4', '0030.mp4', '0031.mp4', '0032.mp4', '0033.mp4', '0034.mp4', '0035.mp4', '0036.mp4', '0037.mp4', '0038.mp4', '0040.mp4', '0041.mp4']


In [16]:
real_videos = [f for f in os.listdir(directory_path+'/real data') if f.endswith(('.mp4'))]

In [18]:
class DeepFakeVideoDataset(Dataset):
    def __init__(self, transform=None, frames_per_video=16):
        self.real_videos = [f for f in os.listdir(directory_path+'/real data') if f.endswith(('.mp4'))]
        self.fake_videos = [f for f in os.listdir(directory_path+'/fake data') if f.endswith(('.mp4'))]
        self.transform = transform
        self.frames_per_video = frames_per_video
        
        # Combine videos and create labels (0 for real, 1 for fake)
        self.video_files = self.real_videos + self.fake_videos
        self.labels = [0] * len(self.real_videos) + [1] * len(self.fake_videos)
    
    def __len__(self):
        return len(self.video_files)
    
    def __getitem__(self, idx):
        video_path = self.video_files[idx]
        label = self.labels[idx]
        
        # Load video and extract frames
        cap = cv2.VideoCapture(video_path)
        frames = []
        count = 0
        
        while count < self.frames_per_video and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convert frame to RGB and resize to 224x224
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (224, 224))
            if self.transform:
                frame = self.transform(frame)
            
            frames.append(frame)
            count += 1
        
        cap.release()
        
        # If the video has fewer frames than required, pad with the last frame
        while len(frames) < self.frames_per_video:
            frames.append(frames[-1])
        
        # Stack frames along the time dimension and convert to tensor
        frames = torch.stack(frames)  # Shape: (frames_per_video, 3, 224, 224)
        return frames, label

In [ ]:
transform = transform.Compose([
    transform.ToTensor(),  # Convert image to PyTorch tensor
    transform.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize using ImageNet mean and std
])